
The following tutorial is an extension from the analysis code used in [ref](https://mne.tools/0.18/auto_tutorials/sample-datasets/plot_sleep.html)

# Research Question


Given two subjects from the Sleep Physionet dataset, namely Alice and Bob, how well can we predict the sleep stages of Bob from Alice's data?

This problem is tackled as supervised multiclass classification task. The aim
is to predict the sleep stage from 5 possible stages for each chunk of 30
seconds of data.

# Initial Setup

In [ ]:
 !pip freeze # Preinstall package

In [ ]:
pip install mne_features

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import mne
#from mne.datasets.sleep_physionet.age import fetch_data
from mne.decoding import (Vectorizer)

from mne_features.feature_extraction import FeatureExtractor  # Take some time because of Numba
from mne_features.feature_extraction import extract_features

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline

In [ ]:
# from mne_features.utils import (_idxiter, power_spectrum, _embed, _get_feature_funcs,
#                     _get_feature_func_names, _psd_params_checker)


In [ ]:
# print(_get_feature_func_names(__name__))

# Load the data
------------



1.   Download sleep data from Sleep Physionet Dataset
2.   Read the downloaded file as raw
3.   Extract the annotation from the raw file
4.   Create epochs of 30 sec from the continous signal

In [ ]:
event_id = {
    'Sleep stage W': 1,
    'Sleep stage 1': 2,
    'Sleep stage 2': 3,
    'Sleep stage 3': 4,
    'Sleep stage 4': 4,
    'Sleep stage R': 5}

dataPath = ['sleep_cassette/SC4001E0-PSG.edf', 'sleep_cassette/SC4011E0-PSG.edf'] #Alice, Bob
annoPath = ['sleep_cassette/SC4001EC-Hypnogram.edf', 'sleep_cassette/SC4011EH-Hypnogram.edf']

def some_operation(dpath,aPath):

    # Read the PSG data
    raw = mne.io.read_raw_edf(dpath, stim_channel='marker', misc=['rectal'])

    # Select only EEG
    raw.drop_channels(['EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker'])

    scalings = dict(eeg=40e-5)
    raw.plot(duration=60,scalings=scalings,remove_dc=False)
    tmax = 30. - 1. / raw.info['sfreq']  # Epoch size
    # print(tmax)

    # Extract the annotation from the raw file
    annot = mne.read_annotations(aPath)
    annot.crop(annot[1]['onset'] - 30 * 60, annot[-2]['onset'] + 30 * 60)

    raw.set_annotations(annot, emit_warning=False)
    events, _ = mne.events_from_annotations(raw,event_id=event_id,chunk_duration=30.)
    # u, indices = np.unique(annot['description'], return_index=True)

    # Create epochs of 30 sec from the continous signal
    epochs = mne.Epochs(raw=raw,
                        events=events,
                        event_id=event_id,
                        tmin=0.,
                        tmax=tmax,
                        baseline=None)

    return epochs

In [ ]:
# Read the PSG data and Hypnograms to create a raw object
epochs_alice = some_operation(dataPath[0],annoPath[0])
epochs_bob = some_operation(dataPath[1],annoPath[1])

In [ ]:
# print(epochs_alice)
epochs_alice.info

# Power Spectral Density

   Visualize Alice vs. Bob PSD by sleep stage.


In [ ]:
stage_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

fig, (ax1, ax2) = plt.subplots(ncols=2,figsize=(15,6))

# # iterate over the subjects
stages = sorted(event_id.keys())
for ax, title, epochs in zip([ax1, ax2],['Alice', 'Bob'],[epochs_alice,epochs_bob]):
    for stage, color in zip(stages, stage_colors):
        epochs[stage].plot_psd(area_mode=None, color=color, ax=ax,fmin=0.1, fmax=40., 
                               show=False,average=True, spatial_colors=False)
    ax.set(title=title, xlabel='Frequency (Hz)')
    
ax1.set(ylabel='µV^2/Hz (dB)')    
ax2.set(ylabel='µV^2/Hz (dB)')
ax2.legend(ax2.lines[2::3], stages)
plt.tight_layout()
plt.show()

# Feature Engineering

The rest of this section we will create EEG features based on relative power
in specific frequency bands to capture this difference between the sleep
stages in our data.

## Custom Function 

In [ ]:
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5]
        Transformed data.
    """
    # specific frequency bands
    FREQ_BANDS = {"delta": [0.5, 4.5],
                  "theta": [4.5, 8.5],
                  "alpha": [8.5, 11.5],
                  "sigma": [11.5, 15.5],
                  "beta": [15.5, 30]}

    psds, freqs = mne.time_frequency.psd_welch(epochs, picks='eeg', fmin=0.5, fmax=30.)
    
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

## Feature Extracted From mne-feature

In [ ]:
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

  This function takes an ``mne.Epochs`` object and creates EEG features based
  on relative power in specific frequency bands that are compatible with
  scikit-learn.

  Parameters
  ----------
  epochs : Epochs
      The data.

  Returns
  -------
  X : numpy array of shape [n_samples, 5]
      Transformed data.
  """
    # specific frequency bands
    FREQ_BANDS = {
        "delta": [0.5, 4.5],
        "theta": [4.5, 8.5],
        "alpha": [8.5, 11.5],
        "sigma": [11.5, 15.5],
        "beta": [15.5, 30]
    }

    #
    selected_features = ['pow_freq_bands']

    freq_bands = np.unique(
        np.concatenate(list(map(list, (FREQ_BANDS.values())))))

    funcs_params = dict(pow_freq_bands__normalize=False,
                        pow_freq_bands__ratios='all',
                        pow_freq_bands__psd_method='fft',
                        pow_freq_bands__freq_bands=freq_bands)

    sfreq = epochs.info['sfreq']
    features_all = extract_features(epochs.get_data(),
                                    sfreq,
                                    selected_funcs=selected_features,
                                    return_as_df=True,
                                    funcs_params=funcs_params)

    return features_all.to_numpy()

List of feature



1.   Bivariate


* max_cross_correlation, 
* Maximum linear cross-correlation, 
* Phase Locking Value, 
* Measure of nonlinear interdependence, 
* Correlation Coefficients, 
* Correlation Coefficients

2.   Univariate

* Slope of a 1D least-squares regression, 
* Mean of the data (per channel), 
* Variance of the data (per channel), 
* Standard deviation of the data, 
* Peak-to-peak (PTP) amplitude of the data (per channel), 
* Skewness of the data (per channel), 
* Kurtosis of the data (per channel), 
* Root-mean squared value of the data (per channel), 
* Quantile of the data (per channel), 
* Hurst exponent of the data (per channel),
* Approximate Entropy (AppEn, per channel),
* Sample Entropy (SampEn, per channel), 
* Decorrelation time (per channel),
* Power Spectrum (computed by frequency bands), 
* Hjorth mobility (per channel),
* Hjorth complexity (per channel), 
* Hjorth mobility (per channel), 
* Hjorth complexity (per channel), 
* Higuchi Fractal Dimension (per channel), 
* Katz Fractal Dimension (per channel), 
* Number of zero-crossings (per channel), 
* Line length (per channel), 
* Spectral Entropy (per channel), 
* SVD entropy (per channel), 
* Linear regression of the the log-log frequency-curve (per channel), 
SVD Fisher Information (per channel), 
* Band energy (per channel), 
* Spectal Edge Frequency (per channel), 
* Energy of Wavelet decomposition coefficients (per channel)

# Multiclass classification workflow using scikit-learn and mne-feature

- `Pipeline`

is just an abstract notion, it's not some existing ml algorithm. Often in ML tasks you need to perform sequence of different transformations (find set of features, generate new features, select only some good features) of raw dataset before applying final estimator.

- `FunctionTransformer`

some class that have fit and transform method, or fit_transform method.
- `eeg_power_band`



## Custom function approach

In [ ]:
from sklearn.preprocessing import FunctionTransformer
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# Pipeline
# FunctionTransformer

"""
1) Extract Feature from the eeg_power_band
2) fit and predict method using random forest
"""

pipe = make_pipeline(FunctionTransformer(eeg_power_band, validate=False),RandomForestClassifier(n_estimators=100, random_state=42))


epochs_train,epochs_test=epochs_alice,epochs_bob
y_train = epochs_alice.events[:, 2]
pipe.fit(epochs_alice, y_train)

# Test
y_pred = pipe.predict(epochs_bob)
# epochs.get_data ()
# Assess the results


y_test = epochs_bob.events[:, 2]
acc = accuracy_score(y_test, y_pred)

print("Accuracy score: {}".format(acc))
# Further analysis of the data
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# We can check the confusion matrix or the classification report.

print(confusion_matrix(y_test, y_pred))

##############################################################################

print(classification_report(y_test, y_pred,target_names={'Sleep stage W': 1,
  'Sleep stage 1': 2,
  'Sleep stage 2': 3,
  'Sleep stage 3/4': 4,
  'Sleep stage R': 5}))

## FeatureExtractor Approach

In [ ]:
FREQ_BANDS = {"delta": [0.5, 4.5],"theta": [4.5, 8.5],"alpha": [8.5, 11.5], "sigma": [11.5, 15.5],"beta": [15.5, 30]}


selected_features = ['pow_freq_bands']

freq_bands=np.unique(np.concatenate(list(map(list, (FREQ_BANDS.values())))))

# raw_testx = mne.io.read_raw_edf(all_data[0][0], stim_channel='marker',misc=['rectal'])
# sfreq=raw_testx.info['sfreq']

sfreq=100

In [ ]:
# from mne_features.feature_extraction import FeatureExtractor  # Take some time because of Numba

# selected_funcs = ['line_length', 'kurtosis', 'ptp_amp', 'skewness','pow_freq_bands']
from sklearn.pipeline import Pipeline
selected_funcs = ['pow_freq_bands']
funcs_params = dict ( pow_freq_bands__normalize=False,
                     pow_freq_bands__ratios='all',
                     pow_freq_bands__psd_method='fft',
                     pow_freq_bands__freq_bands=freq_bands)



# FeatureExtractor >> function under mne-feature
pipe = make_pipeline( FeatureExtractor(sfreq=sfreq,params=funcs_params,
                                       selected_funcs=selected_funcs),
                    Vectorizer(),
                    RandomForestClassifier(n_estimators=100, random_state=42))



# epochs_train,epochs_test=epochs_alice,epochs_bob
pipe.fit(epochs_alice.get_data (), epochs_alice.events[:, 2])

# Test
y_pred = pipe.predict(epochs_bob.get_data ())

# Assess the results
y_test = epochs_bob.events[:, 2]
acc = accuracy_score(y_test, y_pred)

print("Accuracy score: {}".format(acc))
# Further analysis of the data
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# We can check the confusion matrix or the classification report.

print(confusion_matrix(y_test, y_pred))


print(classification_report(y_test, y_pred,target_names={'Sleep stage W': 1,
                                                          'Sleep stage 1': 2,
                                                          'Sleep stage 2': 3,
                                                          'Sleep stage 3/4': 4,
                                                          'Sleep stage R': 5}))

In [ ]:
# pipe = Pipeline([('fe', FeatureExtractor(sfreq=sfreq,params=funcs_params,
#                                        selected_funcs=selected_funcs)),('vectorise',Vectorizer()),('clf',  RandomForestClassifier(n_estimators=100, random_state=42))])
